# Libraries

In [15]:
# General
import pandas as pd
import numpy as np

# Data Management
from dataclasses import dataclass, field
from collections import namedtuple
from typing import List, Any
import pickle

# Graphing
import matplotlib.pyplot as plt

# Data Import

In [16]:
def import_smhi(path, skiprows, usecols, rename):
    df = pd.read_csv(
        path,
        skiprows = skiprows, sep = ";", parse_dates = True,
        usecols = usecols
        )
    df.columns = rename
    df["date"] = pd.to_datetime(df["date"]) # Convert to datetimes
    return df

def import_fmi(path, skiprows, usecols, rename):
    #converters = {col: convert_dashes for col in usecols}  # Apply convert_dashes to each column in usecols
    df = pd.read_csv(
        path,
        skiprows = skiprows, sep = ",", parse_dates = True,
        usecols = usecols,
    #    converters = converters
    na_values = "-"
        )
    df.columns = rename
    #df["date"] = pd.to_datetime(df[["year", "month", "day"]]) # Convert to datetimes
    return df

## Import

In [17]:
# Arctic Atmospheric Indices (NAO and AO)
nao = pd.read_csv("../data/atmosphere/nao/norm.daily.nao.cdas.z500.19500101_current.csv").rename(columns = {"nao_index_cdas": "nao"})
ao = pd.read_csv("../data/atmosphere/ao/norm.daily.ao.cdas.z1000.19500101_current.csv").rename(columns = {"ao_index_cdas": "ao"})

# SMHI - Swedish Meteorological Institute Data for Abisko-Stordalen Palsa Bog
# We're importing both the manual and automatic station data. Why? The auto station data has lots of data gaps but runs to present day. The manual has less data gaps but stops in about 2022. We'll merge them in a bit.
# Quality Flag:
# Green (G) = Checked and approved values.
# Yellow (Y) = Suspicious or aggregated values. Roughly checked archival data and unchecked real-time data (last 2 hours).

se_sto_precip_manual = import_smhi(
    path = "../data/climate/abisko-stordalen-palsa-bog/precipitation/smhi-opendata_5_188800_20250418_163156.csv",
    skiprows = 9,
    usecols = ["Representativt dygn", "Nederbördsmängd", "Kvalitet"],
    rename = ["date", "precipitation", "quality_flag"]
)
se_sto_precip_auto = import_smhi(
    path = "../data/climate/abisko-stordalen-palsa-bog/precipitation/smhi-opendata_5_188790_20250418_163211.csv",
    skiprows = 9,
    usecols = ["Representativt dygn", "Nederbördsmängd", "Kvalitet"],
    rename = ["date", "precipitation", "quality_flag"]
)
se_sto_temp_manual = import_smhi(
    path = "../data/climate/abisko-stordalen-palsa-bog/temperature/smhi-opendata_2_188800_20250418_130808.csv",
    skiprows = 9,
    usecols = ["Representativt dygn", "Lufttemperatur", "Kvalitet"],
    rename = ["date", "temperature", "quality_flag"]
)
se_sto_temp_auto = import_smhi(
    path = "../data/climate/abisko-stordalen-palsa-bog/temperature/smhi-opendata_2_188790_20250418_130625.csv",
    skiprows = 9,
    usecols = ["Representativt dygn", "Lufttemperatur", "Kvalitet"],
    rename = ["date", "temperature", "quality_flag"]
)
se_sto_snowdepth_manual = import_smhi(
    path = "../data/climate/abisko-stordalen-palsa-bog/snowdepth/smhi-opendata_8_40_188800_20250418_164039.csv",
    skiprows = 9,
    usecols = ["Datum", "Snödjup", "Kvalitet"],
    rename = ["date", "snowdepth", "quality_flag"]
)
se_sto_snowdepth_auto = import_smhi(
    path = "../data/climate/abisko-stordalen-palsa-bog/snowdepth/smhi-opendata_8_40_188790_20250418_164059.csv",
    skiprows = 11,
    usecols = ["Datum", "Snödjup", "Kvalitet"],
    rename = ["date", "snowdepth", "quality_flag"]
)

# FMI - Finnish Meteorological Institute Data for Kenttarova, Sodankyla, and Varrio
fi_ken_clim = import_fmi(
    path = "../data/climate/kenttarova/Kittilä Kenttärova_ 1.1.1970 - 14.5.2025.csv",
    skiprows = 0,
    usecols = ["Year", "Month", "Day", "Precipitation amount [mm]", "Snow depth [cm]", "Average temperature [°C]"],
    rename = ["year", "month", "day", "precipitation", "snowdepth", "temperature"]
    )
fi_sod_clim = import_fmi(
    path = "../data/climate/sodankyla/Sodankylä Tähtelä_ 1.1.1891 - 14.5.2025.csv",
    skiprows = 0,
    usecols = ["Year", "Month", "Day", "Precipitation amount [mm]", "Snow depth [cm]", "Average temperature [°C]"],
    rename = ["year", "month", "day", "precipitation", "snowdepth", "temperature"]
    )
fi_var_clim = import_fmi(
    path = "../data/climate/varrio/Salla Värriötunturi_ 1.1.1971 - 14.5.2025.csv",
    skiprows = 0,
    usecols = ["Year", "Month", "Day", "Precipitation amount [mm]", "Snow depth [cm]", "Average temperature [°C]"],
    rename = ["year", "month", "day", "precipitation", "snowdepth", "temperature"]
    )

## Prepare

Functions

In [18]:
# Clean data
def cleaner(df, variable_name):
    df[variable_name] = df[variable_name].replace([-9999.0, "Y"], np.nan) # Replace visible/known missing values
    df = df.reindex(pd.date_range(start = df.index[0], end = df.index[-1], freq = "D")) # Reindex to deal with hidden missing values
    #df.index.name = "date" # put the name of the index column back!
    df = df[[variable_name]].copy() # Select Variables
    return df

def prep_smhi(df_manual, df_auto):
    df_manual = df_manual.set_index("date", drop = False) # Make index
    df_manual.rename_axis(None, inplace = True) # Remove the index column header (avoids column header name confusion)
    df_manual = pd.concat([cleaner(df_manual[[column]].copy(), column) for column in df_manual.columns], axis = 1) # Clean and reassign dataframe

    df_auto = df_auto.set_index("date", drop = False)
    df_auto.rename_axis(None, inplace = True) # Remove the index column header (avoids column header name confusion)
    df_auto = pd.concat([cleaner(df_auto[[column]].copy(), column) for column in df_auto.columns], axis = 1) # Clean and reassign dataframe

    return df_manual.combine_first(df_auto)

def prep_fmi(df):
    # Combine into datetime
    df["date"] = pd.to_datetime(df[["year", "month", "day"]])
    df = df[["date", "precipitation", "temperature", "snowdepth"]]
    
    # Different variables are taken at different times on the same day. This means our dataset has multiple rows for each day so we merge each row based on date, overwriting any NaNs. 
    df = df.groupby("date", as_index = False).agg(lambda x: x.dropna().iloc[0] if not x.dropna().empty else np.nan)
    
    # Make date the index
    df = df.set_index("date", drop = False) 
    df.rename_axis(None, inplace = True) # Remove the index column header (avoids column header name confusion)

    # Clean
    df = pd.concat([cleaner(df[[column]].copy(), column) for column in df.columns], axis = 1) # Clean and reassign dataframe

    # Clip to FluxSat date range
    #df = df.loc[(df["date"] >= fluxsat_start) & (df["date"] < fluxsat_end)]
    
    return df

Clean and Merge NAO and AO

In [19]:
# Make date column (as a datetime object)
nao['date'] = pd.to_datetime(nao[['year', 'month', 'day']])
ao['date'] = pd.to_datetime(ao[['year', 'month', 'day']])

# Merge on date column and select only the date and index columns to avoid duplicates
atmos = nao[["date", "nao"]].merge(right = ao[["date", "ao"]], left_on = "date", right_on = "date")

# Make date the index
atmos = atmos.set_index("date", drop = False) 
atmos.rename_axis(None, inplace = True) # Remove the index column header (avoids column header name confusion)

# This data is snazzy and nicely prepared so we don't need to clean it (huzzah!)

atmos

,date,nao,ao
1950-01-01,1950-01-01,0.365000,-2.511443
1950-01-02,1950-01-02,0.096000,-1.505500
1950-01-03,1950-01-03,-0.416000,-1.173238
1950-01-04,1950-01-04,-0.616000,-1.250540
1950-01-05,1950-01-05,-0.261000,-0.125743
...,...,...,...
2025-04-03,2025-04-03,-0.456126,0.338938
2025-04-04,2025-04-04,-0.527314,-0.273034
2025-04-05,2025-04-05,-0.634172,-0.722893
2025-04-06,2025-04-06,-0.598284,-0.835127


Clean and Merge SMHI data for Abisko-Stordalen Palsa Bog

In [20]:
# Add index, clean data (by "Y" quality flag), and union the cleaned manual and auto dataframes
se_sto_precip = prep_smhi(se_sto_precip_manual, se_sto_precip_auto)
se_sto_temp = prep_smhi(se_sto_temp_manual, se_sto_temp_auto)
se_sto_snowdepth = prep_smhi(se_sto_snowdepth_manual, se_sto_snowdepth_auto)

# Merge SMHI data on date and select only the climate variables
se_sto_clim = se_sto_precip[["date", "precipitation"]].merge(right = se_sto_temp[["date", "temperature"]], left_on = "date", right_on = "date").merge(right = se_sto_snowdepth[["date", "snowdepth"]], left_on = "date", right_on = "date")
se_sto_clim = se_sto_clim.set_index("date", drop = False) # Make index

# Reclean just in case we have a few missing dates (mainly for snowdepth)
se_sto_clim = pd.concat([cleaner(se_sto_clim[[column]].copy(), column) for column in se_sto_clim.columns], axis = 1) # Clean and reassign dataframe

se_sto_clim

C:\Users\Laurie\AppData\Local\Temp\ipykernel_2032\3317781567.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[variable_name] = df[variable_name].replace([-9999.0, "Y"], np.nan) # Replace visible/known missing values
C:\Users\Laurie\AppData\Local\Temp\ipykernel_2032\3317781567.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[variable_name] = df[variable_name].replace([-9999.0, "Y"], np.nan) # Replace visible/known missing values


,date,precipitation,temperature,snowdepth
1913-01-13,1913-01-13,0.0,-14.0,NaN
1913-01-14,1913-01-14,0.1,-19.2,NaN
1913-01-15,1913-01-15,0.0,-20.3,NaN
1913-01-16,1913-01-16,0.1,-19.9,NaN
1913-01-17,1913-01-17,0.0,-10.3,NaN
...,...,...,...,...
2024-12-27,2024-12-27,5.0,1.5,0.58
2024-12-28,2024-12-28,0.4,2.1,0.56
2024-12-29,2024-12-29,0.5,-1.5,0.55
2024-12-30,2024-12-30,0.0,-2.3,0.57


Clean FMI Data for Kenttarova, Sodankyla, and Varrio

In [21]:
fi_ken_clim = prep_fmi(fi_ken_clim)
fi_sod_clim = prep_fmi(fi_sod_clim)
fi_var_clim = prep_fmi(fi_var_clim)

# Preview

In [ ]:
plt.plot(se_sto_clim["precipitation"].resample("MS").mean().rolling(12, center = True, min_periods = 8).mean()) # se_sto_clim

# Export

In [ ]:
atmos.to_csv(f"../data/processed/atmos.csv")
se_sto_clim.to_csv(f"../data/processed/se_sto_climate.csv")
fi_ken_clim.to_csv(f"../data/processed/fi_ken_climate.csv")
fi_sod_clim.to_csv(f"../data/processed/fi_sod_climate.csv")
fi_var_clim.to_csv(f"../data/processed/fi_var_climate.csv")